In [1]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    import numpy as np
    import pandas as pd
    from sklearn.ensemble import RandomForestRegressor
    from sklearn import pipeline, grid_search
    from sklearn.base import BaseEstimator, TransformerMixin
    from sklearn.pipeline import FeatureUnion
    from sklearn.decomposition import TruncatedSVD
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics import mean_squared_error, make_scorer
    from nltk.stem.porter import *
    stemmer = PorterStemmer()
    import re
    import random
    random.seed(2016)


C:\Users\sneha\Anaconda324\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


## Importing the dataset

In [2]:
df_train = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/product_descriptions.csv')
df_attr = pd.read_csv('C:/Users/sneha/Google Drive/Github/DataSet/Kaggle/data/attributes.csv')
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})

In [4]:
df_train = pd.read_excel('pre_processed_data\df_train.xlsx')
df_test = pd.read_excel('pre_processed_data\df_test.xlsx')
df_prod = pd.read_excel('pre_processed_data\df_prod.xlsx')
df_brand = pd.read_excel('pre_processed_data\df_brand.xlsx')
df_attr = pd.read_excel('pre_processed_data\df_attr.xlsx')

## Creating new Dataframes 

In [7]:

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_prod, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')

In [8]:
num_train = df_train.shape[0]

In [107]:

df_train2 = df_train[['id','product_title']]
df_train2.head(1)

,id,product_title
0,2,simpson strong tie 12 gaug angl


In [108]:
df_all = pd.merge(df_all, df_train2, how='left', on='id')

## Pre-processing

In [9]:
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}

f = open('spelling.txt','r')
zspell = {}
for line in f:
    a, b = line.strip("\n").split("|")
    zspell[a]=b
f.close()

In [10]:
import string
from nltk.corpus import stopwords
def text_process(s):
    """
    Takes in a string of text, then performs the following:
    1. Clean up data
    2. Remove all stopwords
    3. Returns a list of the cleaned text  
    
    """
    
    s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
    s = re.sub(r"([a-z])([A-Z])", r"\1 \2", s) #Split words with a A
    s = s.lower()
    s = s.replace("  "," ")
    s = re.sub(r"([0-9])( *),( *)([0-9])", r"\1\4", s)
    s = s.replace(","," ")
    s = s.replace("$"," ")
    s = s.replace("?"," ")
    s = s.replace("-"," ")
    s = s.replace("//","/")
    s = s.replace("..",".")
    s = s.replace(" / "," ")
    s = s.replace(" \\ "," ")
    s = s.replace("."," . ")
    s = s.replace("   "," ")
    s = s.replace("  "," ").strip(" ")
    s = re.sub(r"(.*)\.$", r"\1", s) #end period
    s = re.sub(r"(.*)\/$", r"\1", s) #end period
    s = re.sub(r"^\.(.*)", r"\1", s) #start period
    s = re.sub(r"^\/(.*)", r"\1", s) #start slash
    s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
    s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
    s = s.replace(" x "," xbi ")
    s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
    s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
    s = s.replace("*"," xbi ")
    s = s.replace(" by "," xbi ")
    s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
    s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
    s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
    s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
    s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
    s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
    s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
    s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
    s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
    s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
    s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
    s = s.replace(" v "," volts ")
    s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
    s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
    s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
    s = s.replace("  "," ")
    s = s.replace(" . "," ")   
    s = (" ").join([z for z in s.split(" ") if z not in stopwords.words('english')])
    s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
    s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
    s = s.lower()
    s = (" ").join([str(zspell[z]) if z in zspell else z for z in s.split(" ")])
    return s

In [9]:
df_all['search_term'] = df_all['search_term'].apply(text_process)
df_all['product_title'] = df_all['product_title'].apply(text_process)
df_all['product_description'] = df_all['product_description'].apply(text_process)

In [20]:
def text_stem(s):
    if isinstance(s, str):
        s = s.replace(","," ")
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = s.replace("   "," ")
        s = s.replace("  "," ").strip(" ")
        s = (" ").join([z for z in s.split(" ") if z not in stopwords.words('english')])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        s = s.lower()       
        return s
    else:
        return "null"

In [21]:

df_all['brand'] = df_all['brand'].map(lambda x:text_stem(x))

In [48]:
df_all = df_all.drop(['product_titlee_num'], axis = 1)
df_all.head()

,id,product_title,product_title_str,product_uid,relevance,search_term,search_term_num,search_term_str,product_description,product_description_num,product_description_str,brand,len_of_query,len_of_title,len_of_description,len_of_brand,prod_desc_score,prod_desc_score_str,prod_desc_score_num
0,2,0,0,100001,3.00,angl bracket,NaN,angl bracket,angl make joint stronger also provid consist s...,90.0 9.0 3.0 1.0 12.0 10.0 1.0 1.0,angl make joint stronger also provid consist s...,simpson strong-ti,2,6,96,2,3,3,0
1,3,0,0,100001,2.50,l bracket,NaN,l bracket,angl make joint stronger also provid consist s...,90.0 9.0 3.0 1.0 12.0 10.0 1.0 1.0,angl make joint stronger also provid consist s...,simpson strong-ti,2,6,96,2,0,0,0
2,9,0,0,100002,3.00,deck,NaN,deck,behr premium textur deckov innov solid color c...,2.0,behr premium textur deckov innov solid color c...,behr premium textur deckov,1,11,124,4,3,3,0
3,16,0,0,100005,2.33,rain shower head,NaN,rain shower head,updat bathroom delta vero singl handl shower f...,3.0 10000.0,updat bathroom delta vero singl handl shower f...,delta,3,11,77,1,1,1,0
4,17,0,0,100005,2.67,shower faucet,NaN,shower faucet,updat bathroom delta vero singl handl shower f...,3.0 10000.0,updat bathroom delta vero singl handl shower f...,delta,2,11,77,1,2,2,0


## Building features

In [19]:
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(str(x).split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(str(x).split())).astype(np.int64)

In [20]:
def isFloat(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

def eval_f(expr):
    first = ''
    second = ""
    flag = 0

    if expr.find("/") == 1:    
        for a in expr:            
            if a == "/":
                flag = 1
                continue
            if flag == 0:
                first += a
            if flag == 1:
                second += a
        
        if (first.isnumeric() == True and  second.isnumeric() == True):
            if (float(second)!= 0):
                return (float(first)/float(second))
            else:
                return 1000
    else:
        if (expr.isnumeric()):
            return float(expr)
        else:
            return 1

def multiply_search(exp):
    if isFloat(exp):
        return 0
    a = exp.split()
    
    prod = 0
    for i in range(0,len(a)):
        if a[i] == 'xbi':        
            
            if (i+1 != len(a)):
                #print (a[i-1],a[i+1])
                try:
                    prod = eval_f(re.sub("[a-z.]", "",a[i-1])) * eval_f(re.sub("[a-z.]", "",a[i+1]))
                except Exception as e:
                    pass
                
                a[i] = str(prod)
                #print(prod)
   
    return (" ".join(a))   

def to_float(exp):
    if isinstance(exp, str): 
        s = (" ").join([str(float(z)) for z in exp.split(" ") if isFloat(z) ])        
        return s
def to_str(exp):
    if isinstance(exp, str): 
        s = (" ").join([z for z in exp.split(" ") if not isFloat(z) ])
        return s

In [28]:
df_all['search_term'] = df_all['search_term'].apply(multiply_search)
df_all['search_term_num'] = df_all['search_term'].apply(to_float)
df_all['search_term_str'] = df_all['search_term'].apply(to_str)



In [29]:
df_all['product_description'] = df_all['product_description'].apply(multiply_search)
df_all['product_description_num'] = df_all['product_description'].apply(to_float)
df_all['product_description_str'] = df_all['product_description'].apply(to_str)

In [37]:
def prod_title_score(description, search_term):    
        
    prod_title_list = str(description).split()
    total_len = len(str(search_term).split())
    count = 0
    for word in str(search_term).split():
        count += prod_title_list.count(word)            

    return count


In [38]:
df_all['prod_desc_score'] = df_all.apply(lambda df_all: prod_title_score(df_all['product_description'],df_all['search_term'] ) ,axis = 1)
df_all['prod_desc_score_str'] = df_all.apply(lambda df_all: prod_title_score(df_all['product_description_str'],df_all['search_term_str'] ) ,axis = 1)
df_all['prod_desc_score_num'] = df_all.apply(lambda df_all: prod_title_score(df_all['product_description_num'],df_all['search_term_num'] ) ,axis = 1)

In [51]:
df_all['product_title_score'] = df_all.apply(lambda df_all: prod_title_score(df_all['product_title'],df_all['search_term'] ) ,axis = 1)
df_all['product_title_str'] = df_all.apply(lambda df_all: prod_title_score(df_all['product_title_str'],df_all['search_term_str'] ) ,axis = 1)
#df_all['product_title_num'] = df_all.apply(lambda df_all: prod_title_score(df_all['product_title_num'],df_all['search_term_num'] ) ,axis = 1)

In [52]:
df_all.head(5)

,id,product_title,product_title_str,product_uid,relevance,search_term,search_term_num,search_term_str,product_description,product_description_num,product_description_str,brand,len_of_query,len_of_title,len_of_description,len_of_brand,prod_desc_score,prod_desc_score_str,prod_desc_score_num
0,2,0,0,100001,3.00,angl bracket,NaN,angl bracket,angl make joint stronger also provid consist s...,90.0 9.0 3.0 1.0 12.0 10.0 1.0 1.0,angl make joint stronger also provid consist s...,simpson strong-ti,2,6,96,2,3,3,0
1,3,0,0,100001,2.50,l bracket,NaN,l bracket,angl make joint stronger also provid consist s...,90.0 9.0 3.0 1.0 12.0 10.0 1.0 1.0,angl make joint stronger also provid consist s...,simpson strong-ti,2,6,96,2,0,0,0
2,9,0,0,100002,3.00,deck,NaN,deck,behr premium textur deckov innov solid color c...,2.0,behr premium textur deckov innov solid color c...,behr premium textur deckov,1,11,124,4,3,3,0
3,16,0,0,100005,2.33,rain shower head,NaN,rain shower head,updat bathroom delta vero singl handl shower f...,3.0 10000.0,updat bathroom delta vero singl handl shower f...,delta,3,11,77,1,1,1,0
4,17,0,0,100005,2.67,shower faucet,NaN,shower faucet,updat bathroom delta vero singl handl shower f...,3.0 10000.0,updat bathroom delta vero singl handl shower f...,delta,2,11,77,1,2,2,0


In [53]:
def seg_words(str1, str2):
    str2 = str2.lower()
    str2 = re.sub("[^a-z0-9./]"," ", str2)
    str2 = [z for z in set(str2.split()) if len(z)>2]
    words = str1.lower().split(" ")
    s = []
    for word in words:
        if len(word)>3:
            s1 = []
            s1 += segmentit(word,str2,True)
            if len(s)>1:
                s += [z for z in s1 if z not in ['er','ing','s','less'] and len(z)>1]
            else:
                s.append(word)
        else:
            s.append(word)
    return (" ".join(s))
def segmentit(s, txt_arr, t):
    st = s
    r = []
    for j in range(len(s)):
        for word in txt_arr:
            if word == s[:-j]:
                r.append(s[:-j])
                #print(s[:-j],s[len(s)-j:])
                s=s[len(s)-j:]
                r += segmentit(s, txt_arr, False)
    if t:
        i = len(("").join(r))
        if not i==len(st):
            r.append(st[i:])
    return r

def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

In [141]:
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))

In [89]:

df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))

In [120]:
df_all['product_info'] = df_all['search_term'].map(str)+"\t"+df_all['product_title'].map(str) +"\t"+df_all['product_description'].map(str)

In [135]:

df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))


df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']

In [ ]:
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
df_all['attr'] = df_all['search_term'].map(str)+"\t"+df_all['brand'].map(str)
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']
df_brand = pd.unique(df_all.brand.ravel())

In [69]:
i = 1000
d={}
for s in df_brand:
    d[s]=i
    i+=3
df_all['brand_feature'] = df_all['brand'].map(lambda x:d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(str(x)))
df_all.to_csv('df_all.csv')
#df_all = pd.read_csv('df_all.csv', encoding="ISO-8859-1", index_col=0)


In [143]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
y_train = df_train['relevance'].values
X_train =df_train[:]
X_test = df_test[:]

In [70]:
X_train.head(1)

,id,product_title,product_title_str,product_uid,relevance,search_term,search_term_num,search_term_str,product_description,product_description_num,...,query_last_word_in_description,word_in_title,word_in_description,ratio_title,ratio_description,attr,word_in_brand,ratio_brand,brand_feature,search_term_feature
0,2,0,0,100001,3,angl bracket,NaN,angl bracket,angl make joint stronger also provid consist s...,90.0 9.0 3.0 1.0 12.0 10.0 1.0 1.0,...,0,0,1,0,0.5,angl bracket\tsimpson strong-ti,0,0,1000,12


In [145]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hd_searches):
        d_col_drops=['id','relevance','search_term','search_term_num','search_term_str','product_title',
                     'product_title_str','product_description','product_description_num','product_description_str',
                    'product_info','attr','brand']
        #d_col_drops=['id','relevance','search_term','product_title','product_description','product_info','attr','brand']
        hd_searches = hd_searches.drop(d_col_drops,axis=1).values
        return hd_searches

class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

In [75]:
X_train = X_train.fillna(0)
X_train.head(5)

,id,product_title,product_title_str,product_uid,relevance,search_term,search_term_num,search_term_str,product_description,product_description_num,...,query_last_word_in_description,word_in_title,word_in_description,ratio_title,ratio_description,attr,word_in_brand,ratio_brand,brand_feature,search_term_feature
0,2,0,0,100001,3.00,angl bracket,0,angl bracket,angl make joint stronger also provid consist s...,90.0 9.0 3.0 1.0 12.0 10.0 1.0 1.0,...,0,0,1,0,0.500000,angl bracket\tsimpson strong-ti,0,0.00,1000,12
1,3,0,0,100001,2.50,l bracket,0,l bracket,angl make joint stronger also provid consist s...,90.0 9.0 3.0 1.0 12.0 10.0 1.0 1.0,...,0,0,1,0,0.500000,l bracket\tsimpson strong-ti,0,0.00,1000,9
2,9,0,0,100002,3.00,deck,0,deck,behr premium textur deckov innov solid color c...,2.0,...,1,0,1,0,1.000000,deck\tbehr premium textur deckov,1,0.25,1003,4
3,16,0,0,100005,2.33,rain shower head,0,rain shower head,updat bathroom delta vero singl handl shower f...,3.0 10000.0,...,0,0,1,0,0.333333,rain shower head\tdelta,0,0.00,1006,16
4,17,0,0,100005,2.67,shower faucet,0,shower faucet,updat bathroom delta vero singl handl shower f...,3.0 10000.0,...,1,0,2,0,1.000000,shower faucet\tdelta,0,0.00,1006,13


In [162]:
rfr = RandomForestRegressor(n_estimators = 500, n_jobs = 1, random_state = 2016, verbose = 1)
tfidf = TfidfVectorizer(ngram_range=(1, 1),min_df=1)
tsvd = TruncatedSVD(n_components=10, random_state = 2016)
clf = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                        ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt2', pipeline.Pipeline([('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)])),
                        ('txt3', pipeline.Pipeline([('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)])),
                        ('txt4', pipeline.Pipeline([('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)]))
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 0.5,
                        'txt2': 0.25,
                        'txt3': 0.0,
                        'txt4': 0.5
                        },
                #n_jobs = -1
                )), 
        ('rfr', svr_rbf)])
param_grid = {'rfr__max_features': [10], 'rfr__max_depth': [20]}
model = grid_search.GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs = 1, cv = 2, verbose = 20, scoring=RMSE)


In [163]:
model.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] rfr__max_depth=20, rfr__max_features=10 .........................


ValueError: Invalid parameter max_depth for estimator SVR

In [149]:
print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)
print(model.best_score_ + 0.47003199274)

Best parameters found by grid search:
{'rfr__max_depth': 20, 'rfr__max_features': 10}
Best CV score:
-0.47016679181
-0.000134799069826


In [150]:
y_pred = model.predict(X_test)
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission_tfidf2.csv',index=False)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    1.1s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:    4.7s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:   10.8s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   12.1s finished


In [159]:
model.grid_scores_


<bound method BaseEstimator.get_params of GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('cst', cust_regression_vals()), ('txt1', Pipeline(steps=[('s1', cust_txt_col(key='search_term')), ('tfidf1', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', ...imators=500, n_jobs=1, oob_score=False, random_state=2016,
           verbose=1, warm_start=False))]),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'rfr__max_depth': [20], 'rfr__max_features': [10]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring=make_scorer(fmean_squared_error, greater_is_better=False),
       verbose=20)>

In [161]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)

In [133]:
df_all['product_info'].head(5).map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))

0    1
1    1
2    1
3    1
4    2
Name: product_info, dtype: int64

In [137]:

df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']